# Somministrazione vaccini

In [1]:
import pandas as pd
import numpy as np
import datetime
import pickle as pkl

import matplotlib.pyplot as plt
import seaborn as sns
from aidam.math_utils import moving_function_rev1
#from aidam.miscellanea_utils import find_matlabstyle
#from aidam.aicharts import get_subplot_cells_sequence, confplot_ax, auto_grid_shape

import plotly.graph_objects as go

import warnings
warnings.filterwarnings('ignore')

## Parametri

In [2]:
data_file='https://raw.githubusercontent.com/italia/covid19-opendata-vaccini/master/dati/somministrazioni-vaccini-summary-latest.csv'

## Lettura dati e preprocessing

In [3]:
data=pd.read_csv(data_file)
print(data.columns)

In [ ]:
data.head(5)

,data_somministrazione,area,totale,sesso_maschile,sesso_femminile,prima_dose,seconda_dose,pregressa_infezione,dose_addizionale_booster,codice_NUTS1,codice_NUTS2,codice_regione_ISTAT,nome_area
0,2021-02-12,PAB,731,217,514,465,248,18,0,ITH,ITH1,4,Provincia Autonoma Bolzano / Bozen
1,2021-03-23,PIE,12978,5482,7496,6363,6256,359,0,ITC,ITC1,1,Piemonte
2,2021-07-14,MOL,3447,1686,1761,211,3218,18,0,ITF,ITF2,14,Molise
3,2021-09-15,CAM,15313,7572,7741,6068,8546,696,3,ITF,ITF3,15,Campania
4,2021-12-21,SIC,32810,15746,17064,3372,2485,60,26893,ITG,ITG1,19,Sicilia


### Popolazione

In [ ]:
np.unique(data['area'])

array(['ABR', 'BAS', 'CAL', 'CAM', 'EMR', 'FVG', 'LAZ', 'LIG', 'LOM',
       'MAR', 'MOL', 'PAB', 'PAT', 'PIE', 'PUG', 'SAR', 'SIC', 'TOS',
       'UMB', 'VDA', 'VEN'], dtype=object)

In [ ]:
  
popolazione={'ABR': 1293941,
            'BAS': 553254,
            'CAL': 1894110,
            'CAM': 5712143,
            'EMR': 4464119,
            'FVG': 1206216,
            'LAZ': 5755700,
            'LIG': 1524826,
            'LOM': 10027602,
            'MAR': 1512672,
            'MOL': 300516,
            'PIE': 4311217,
            'PUG': 3953305,
            'SAR': 1611621,
            'SIC': 4875290,
            'TOS': 3692555,
            'UMB': 870165,
            'VDA': 125034,
            'VEN': 4879133,
            'PAT': 545425,
            'PAB': 532644}

## Somministrazioni per regione

In [ ]:
regione='TOS'
variabili=['totale','prima_dose','seconda_dose','dose_aggiuntiva']


sub_df=data[data['area']==regione]
sub_df.sort_values(by='data_somministrazione',inplace=True)

fig=go.Figure()
    
    
for v in variabili:
    fig.add_trace(go.Scatter(x=sub_df['data_somministrazione'],
                             y=moving_function_rev1(sub_df[v],np.mean,3,3),
                             mode='lines',
                            name=v))
   
fig.update_layout(
    title='Somministrazioni %s'%regione,
    autosize=False,
    showlegend=True,
    width=1200,
    height=600)
fig.show()

KeyError: 'dose_aggiuntiva'

In [ ]:
ita_df=data.groupby('data_somministrazione').sum()
ita_df.reset_index(inplace=True)
ita_df.tail(10)

,data_somministrazione,totale,sesso_maschile,sesso_femminile,prima_dose,seconda_dose,pregressa_infezione,dose_aggiuntiva,dose_booster,codice_regione_ISTAT
323,2021-11-15,208189,101026,107163,18248,35257,1191,26848,126645,214
324,2021-11-16,214289,101286,113003,18045,29081,1052,28223,137888,214
325,2021-11-17,218908,102466,116442,18097,27056,941,29675,143139,214
326,2021-11-18,237228,109824,127404,20491,28828,882,32962,154065,214
327,2021-11-19,237434,108481,128953,19635,26581,910,33418,156890,214
328,2021-11-20,189188,86155,103033,17732,19718,864,29690,121184,214
329,2021-11-21,81138,37765,43373,8635,10034,356,11653,50460,200
330,2021-11-22,248288,118455,129833,21584,20415,914,39383,165992,214
331,2021-11-23,271883,127238,144645,20786,21983,744,40801,187569,214
332,2021-11-24,271810,127668,144142,10484,19967,706,42045,198608,214


In [ ]:
variabili=['totale','prima_dose','seconda_dose','dose_aggiuntiva']

fig=go.Figure()
    
    
for v in variabili:
    fig.add_trace(go.Scatter(x=ita_df['data_somministrazione'],
                             y=moving_function_rev1(ita_df[v],np.mean,3,3),
                             mode='lines',
                            name=v))
   
fig.update_layout(
    title='Somministrazioni Italia',
    autosize=True,
    showlegend=True,
    width=1200,
    height=600)
fig.show()

## Ciclo completo in rapporto alla popolazione

In [ ]:
data_regionali_cumulativi=data.groupby('area').sum()
data_regionali_cumulativi

,totale,sesso_maschile,sesso_femminile,prima_dose,seconda_dose,pregressa_infezione,dose_aggiuntiva,dose_booster,codice_regione_ISTAT
area,,,,,,,,,
ABR,2058164,1001670,1056494,981568,934962,30708,30588,80338,4238
BAS,874106,427257,446849,425263,400429,11220,8101,29093,5576
CAL,2731988,1337130,1394858,1358875,1260353,21613,24316,66831,5922
CAM,8642859,4218619,4424240,4180405,3874849,159660,123847,304098,4950
EMR,7270691,3540100,3730591,3445427,3245502,141156,76609,361997,2640
FVG,1851451,906018,945433,891311,840224,43247,13917,62752,1992
LAZ,9222441,4409809,4812632,4494111,4070864,139575,27419,490472,3996
LIG,2425527,1168595,1256932,1169024,1105326,23849,11466,115862,2331
LOM,16524390,8055202,8469188,7925420,7424861,286995,82745,804369,999


In [ ]:
rate_seconda_dose={}
rate_prima_dose={}
for reg in np.unique(data['area']):
    rate_seconda_dose[reg]=100*data_regionali_cumulativi.loc[reg]['seconda_dose']/popolazione[reg]
    rate_prima_dose[reg]=100*data_regionali_cumulativi.loc[reg]['prima_dose']/popolazione[reg]

for r in rate_prima_dose.keys():
    print('%s  Prima dose: %4.2f   Seconda dose: %4.2f'%(r,rate_prima_dose[r],rate_seconda_dose[r]))

ABR  Prima dose: 75.86   Seconda dose: 72.26
BAS  Prima dose: 76.87   Seconda dose: 72.38
CAL  Prima dose: 71.74   Seconda dose: 66.54
CAM  Prima dose: 73.18   Seconda dose: 67.84
EMR  Prima dose: 77.18   Seconda dose: 72.70
FVG  Prima dose: 73.89   Seconda dose: 69.66
LAZ  Prima dose: 78.08   Seconda dose: 70.73
LIG  Prima dose: 76.67   Seconda dose: 72.49
LOM  Prima dose: 79.04   Seconda dose: 74.04
MAR  Prima dose: 73.96   Seconda dose: 70.33
MOL  Prima dose: 78.60   Seconda dose: 73.49
PAB  Prima dose: 68.04   Seconda dose: 62.11
PAT  Prima dose: 74.29   Seconda dose: 69.59
PIE  Prima dose: 74.80   Seconda dose: 69.88
PUG  Prima dose: 78.11   Seconda dose: 72.86
SAR  Prima dose: 78.09   Seconda dose: 74.49
SIC  Prima dose: 71.69   Seconda dose: 66.46
TOS  Prima dose: 79.92   Seconda dose: 75.16
UMB  Prima dose: 77.61   Seconda dose: 72.61
VDA  Prima dose: 70.85   Seconda dose: 67.84
VEN  Prima dose: 74.48   Seconda dose: 70.54


In [ ]:
fig=go.Figure()


ordine=np.argsort(list(rate_seconda_dose.values()))

fig.add_trace(go.Bar(x=np.array(list(rate_seconda_dose.keys()))[ordine],
                     y=np.array(list(rate_seconda_dose.values()))[ordine],
                     name='Seconda dose'))

fig.add_trace(go.Bar(x=np.array(list(rate_prima_dose.keys()))[ordine],
                     y=np.array(list(rate_prima_dose.values()))[ordine],
                     name='Prima dose'))

fig.update_layout(
    title='Percentuale vaccinati ciclo completo',
    autosize=False,
    showlegend=True,
    width=1200,
    height=400)

In [ ]:
fig=go.Figure()

ordine=np.argsort(list(rate_prima_dose.values()))

fig.add_trace(go.Bar(x=np.array(list(rate_prima_dose.keys()))[ordine],
                     y=np.array(list(rate_prima_dose.values()))[ordine]))

fig.update_layout(
    title='Percentuale vaccinati prima dose',
    autosize=False,
    showlegend=False,
    width=1200,
    height=400)